In [2]:
import cv2
import dlib
from scipy.spatial import distance
from imutils import face_utils
import playsound
import time
import imutils

alarm_ON = False
model_path = 'data/shape_predictor_68_face_landmarks.dat'
alert_sound = 'sound/digital-alarm-clock-151920.mp3'

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect():
    global alarm_ON
    threshold = 0.25
    frame_check = 10
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(model_path)
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

    cap = cv2.VideoCapture(0)
    flag = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = imutils.resize(frame, width=640, height=640)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        for face in faces:
            shape = predictor(gray, face)
            shape = face_utils.shape_to_np(shape)
            lShape = shape[lStart:lEnd]
            rShape = shape[rStart:rEnd]
            left_ear = eye_aspect_ratio(lShape)
            right_ear = eye_aspect_ratio(rShape)
            ear = (left_ear + right_ear) / 2.0
            leftConvexHull = cv2.convexHull(lShape)
            rightConvexHull = cv2.convexHull(rShape)
            cv2.drawContours(frame, [leftConvexHull], -1, (0, 255, 0), 3)
            cv2.drawContours(frame, [rightConvexHull], -1, (0, 255, 0), 3)

            if ear < threshold:
                flag += 1
                if flag >= frame_check:
                    if not alarm_ON:
                        alarm_ON = True
                        time.sleep(0.1)

                    cv2.putText(frame, '****************Drowsy!****************', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, 'Sleeping Time 00:' + str(flag) + ' sec', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    playsound.playsound(alert_sound, block = True)
            else:
                alarm_ON = False
                flag = 0
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    detect()


: 